REMEMBER: FIRST CREATE A COPY OF THIS FILE WITH A UNIQUE NAME AND DO YOUR WORK THERE. AND MAKE SURE YOU COMMIT YOUR CHANGES TO THE `hw3_submissions` BRANCH.

# Assignment 3 | Cleaning and Exploring Data with Pandas



<img src="data/scoreCard.jpg" width=250>

In this assignment, you will investigate restaurant food safety scores for restaurants in San Francisco. Above is a sample score card for a restaurant. The scores and violation information have been made available by the San Francisco Department of Public Health. 

## Loading Food Safety Data


There are 2 files in the data directory:
1. business.csv containing food establishments in San Francisco
1. inspections.csv containing retaurant inspections records

Let's start by loading them into Pandas dataframes.  One of the files, business.csv, has encoding (ISO-8859-1), so you will need to account for that when reading it.

### Question 1

#### Question 1a
Read the two files noted above into two pandas dataframes named `bus` and `ins`, respectively. Print the first 5 rows of each to inspect them.


In [1]:
import pandas as pd

In [2]:
bus = pd.read_csv('data/businesses.csv', encoding='ISO-8859-1')
ins = pd.read_csv('data/inspections.csv')

In [3]:
bus.head(5)

,business_id,name,address,city,state,postal_code,latitude,longitude,phone_number
0,10,TIRAMISU KITCHEN,033 BELDEN PL,San Francisco,CA,94104,37.791116,-122.403816,+14154217044
1,19,NRGIZE LIFESTYLE CAFE,"1200 VAN NESS AVE, 3RD FLOOR",San Francisco,CA,94109,37.786848,-122.421547,+14157763262
2,24,OMNI S.F. HOTEL - 2ND FLOOR PANTRY,"500 CALIFORNIA ST, 2ND FLOOR",San Francisco,CA,94104,37.792888,-122.403135,+14156779494
3,31,NORMAN'S ICE CREAM AND FREEZES,2801 LEAVENWORTH ST,San Francisco,CA,94133,37.807155,-122.419004,NaN
4,45,CHARLIE'S DELI CAFE,3202 FOLSOM ST,San Francisco,CA,94110,37.747114,-122.413641,+14156415051


In [4]:
ins.head(5)

,business_id,score,date,type
0,10,82,20160503,routine
1,10,94,20140729,routine
2,10,92,20140114,routine
3,19,94,20160513,routine
4,19,94,20141110,routine


## Examining the Business data

From its name alone, we expect the `businesses.csv` file to contain information about the restaurants. Let's investigate this dataset.

### Question 2

#### Question 2a: How many records are there?

In [5]:
bus['business_id'].count()

print("There are " + str(bus['business_id'].count()) + " records.")

#could also do len(bus.index)

There are 6315 records.


#### Question 2b: How many unique business IDs are there?  

In [6]:
unique_values = bus.groupby('business_id')['name'].unique()

# Count the number of unique values in each group
unique_count = unique_values.apply(lambda x: len(x))

# Print the result
#print(unique_count)

print('There are 6315 unique business IDs.')

There are 6315 unique business IDs.


#### Question 2c: What are the 5 most common businesses by name, and how many are there in San Francisco?

In [7]:
n = 5
bus.apply(lambda x: pd.Series(x.value_counts().index[:n]))

,business_id,name,address,city,state,postal_code,latitude,longitude,phone_number
0,10,STARBUCKS COFFEE,24 WILLIE MAYS PLAZA,San Francisco,CA,94110,37.778382,-122.391151,+14159721500
1,75166,PEET'S COFFEE & TEA,OFF THE GRID,NaN,NaN,94103,37.795225,-122.393770,+14152410256
2,75139,MCDONALDS,428 11TH ST,NaN,NaN,94102,37.774722,-122.406761,+14159744040
3,75123,SAN FRANCISCO SOUP COMPANY,2948 FOLSOM ST,NaN,NaN,94107,37.758941,-122.485910,+14158723124
4,75113,WALGREENS,3251 20TH AVE,NaN,NaN,94133,37.808240,-122.410189,+14159891866


In [8]:
print('The 5 most common businesses and the amount for each are as follows:')
bus['name'].value_counts().head(5)


The 5 most common businesses and the amount for each are as follows:


STARBUCKS COFFEE              72
PEET'S COFFEE & TEA           24
MCDONALDS                     12
SAN FRANCISCO SOUP COMPANY    11
WALGREENS                     11
Name: name, dtype: int64

## Zip code

Next, let's  explore some of the variables in the business table. We begin by examining the postal code.

### Question 3

#### Question 3a
How are the zip code values stored in python (i.e. data type)?

To answer this you might want to examine a particular entry.

In [9]:
bus['postal_code'].head()
bus.dtypes
type(bus.loc[0, 'name'])

str

In [10]:

print('The zip code values stored are string.')

The zip code values stored are string.


#### Question 3b

What are the unique values of postal_code?

In [11]:
#this counts the distint values when using nunique
bus['postal_code'].nunique()

46

In [12]:
# shows us the unique values
print('The unique values of postal_code are as follows:')
bus['postal_code'].unique()

The unique values of postal_code are as follows:


array(['94104', '94109', '94133', '94110', '94122', '94103', '94115',
       '94131', '94111', '94117', '94107', '94108', '94102', '94132',
       '94105', '94134', '94124', '94116', '94121', '94118', '94112',
       '94123', '94114', '94127', '941102019', '941', '94545', '94066',
       '941033148', nan, '94158', '95105', '94013', '94130', 'CA',
       '92672', '94120', '94143', '94609', '94101', '00000', '94188',
       '94621', '94014', '94129', '94602', 'Ca'], dtype=object)

#### Question 3c

Let's say we decide to exclude the businesses that have no zipcode for our analysis (which might include food trucks for example).  Use the list of valid 5-digit zip codes below to create a new dataframe called bus_valid, with only businesses whose postal_codes show up in this list of valid zipcodes. How many businesses are there in this new dataframe?

In [13]:
validZip = ["94102", "94103", "94104", "94105", "94107", "94108",
            "94109", "94110", "94111", "94112", "94114", "94115",
            "94116", "94117", "94118", "94121", "94122", "94123", 
            "94124", "94127", "94131", "94132", "94133", "94134"]

In [14]:
bus_valid = bus[bus['postal_code'].isin(validZip)]
bus_valid.shape
print('There are ' + str(len(bus_valid)) + ' businesses in this new dataframe with valid zipcodes.')

There are 5999 businesses in this new dataframe with valid zipcodes.


## Latitude and Longitude

Another aspect of the data we want to consider is the prevalence of missing values. If many records have missing values then we might be concerned about whether the nonmissing values are representative of the population.

### Question 4
 
Consider the longitude and latitude in the business DataFrame. 

#### Question 4a

How many businesses are missing longitude values, working with only the businesses that are in the list of valid zipcodes?

In [15]:
bus_valid[pd.isnull(bus_valid['longitude'])].shape[0]
print('There are ' + str(bus_valid[pd.isnull(bus_valid['longitude'])].shape[0]) 
      + ' businesses that are missing longitude values.')

There are 2483 businesses that are missing longitude values.


#### Question 4b

Create a new dataframe with one row for each valid zipcode. The dataframe should include the following three columns:

1. `postal_code`: Contains the zip codes in the `validZip` variable above.
2. `null_lon`: The number of businesses in that zipcode with missing `longitude` values.
3. `not_null_lon`: The number of businesses without missing `longitude` values.

In [16]:
data = []
for zip in validZip:
    rowsForZip = bus_valid[bus_valid['postal_code'] == zip]
    null_lon = rowsForZip[pd.isnull(rowsForZip['longitude'])].shape[0]
    not_null_lon = rowsForZip[pd.notnull(rowsForZip['longitude'])].shape[0]
    data.append([zip, null_lon, not_null_lon])
vc = pd.DataFrame(data)
vc.columns = ['postal_code','null_lon','not_null_lon']
#new dataframe is vc
vc

,postal_code,null_lon,not_null_lon
0,94102,196,262
1,94103,252,306
2,94104,59,74
3,94105,100,126
4,94107,247,204
5,94108,82,142
6,94109,146,237
7,94110,251,334
8,94111,115,167
9,94112,59,131


#### 4c. Do any zip codes appear to have more than their 'fair share' of missing longitude? 

To answer this, you will want to compute the proportion of missing longitude values for each zip code, and print the proportion missing longitude, and print the top five zipcodes in descending order of proportion missing postal_code.


In [17]:
vc['long_proportion'] = (vc['null_lon'] / (vc['null_lon'] + vc['not_null_lon']))
vc.sort_values(by=['long_proportion'], ascending=False).head(5)

,postal_code,null_lon,not_null_lon,long_proportion
4,94107,247,204,0.547672
18,94124,88,76,0.536585
21,94132,65,69,0.485075
10,94114,108,122,0.469565
16,94122,123,147,0.455556


# Investigate the inspection data

Let's now turn to the inspection DataFrame. Earlier, we found that `ins` has 4 columns, these are named `business_id`, `score`, `date` and `type`.  In this section, we determine the granularity of `ins` and investigate the kinds of information provided for the inspections. 

### Question 5

#### Question 5a
As with the business data, assess whether there is one inspection record for each business, by counting how many rows are in the data and how many unique businesses there are in the data. If they are exactly the same number, it means there is only one inspection per business, clearly.

In [18]:
print(ins.shape[0])
print(ins['business_id'].nunique())
print("There are multiple inspection records for each business.")

15430
5730
There are multiple inspection records for each business.


#### Question 5b

What values does `type` take on? How many occurrences of each value is in the DataFrame? Create a new dataframe named `ins2` by copying `ins` and keeping only records with values of `type` that occur more than 10 times in the original table. In other words, eliminate records that have values of `type` that occur rarely (< 10 times). Check the result to make sure rare types are eliminated.

In [19]:
ins['type'].unique()
print(ins)

       business_id  score      date     type
0               10     82  20160503  routine
1               10     94  20140729  routine
2               10     92  20140114  routine
3               19     94  20160513  routine
4               19     94  20141110  routine
...            ...    ...       ...      ...
15425        89515     98  20161103  routine
15426        89701    100  20161206  routine
15427        89790     96  20161129  routine
15428        89803    100  20161215  routine
15429        89900     93  20161206  routine

[15430 rows x 4 columns]


In [20]:
type(ins.loc[0, 'type'])
print("Type values are routine and complaint, and are stored as string.")

Type values are routine and complaint, and are stored as string.


In [21]:
ins[ins['type'] == 'routine'].shape[0]
ins[ins['type'] == 'complaint'].shape[0]
print("The occurrences for routine are " + str(ins[ins['type'] == 'routine'].shape[0]) +"." )
print("The occurrences for complaint are " + str(ins[ins['type'] == 'complaint'].shape[0]) +"." )

The occurrences for routine are 15429.
The occurrences for complaint are 1.


In [22]:
# new dataframe ins2
ins2 = ins[ins['type'] == 'routine'] 
ins2

,business_id,score,date,type
0,10,82,20160503,routine
1,10,94,20140729,routine
2,10,92,20140114,routine
3,19,94,20160513,routine
4,19,94,20141110,routine
...,...,...,...,...
15425,89515,98,20161103,routine
15426,89701,100,20161206,routine
15427,89790,96,20161129,routine
15428,89803,100,20161215,routine


#### Question 5c

Since the data was stored in a .csv file, the dates are formatted as strings such as `20160503`. Once we read in the data, we would like to have dates in an appropriate format for analysis. Add a new column called `year` by capturing the first four characters of the date column. 

Hint: we have seen multiple ways of doing this in class, includings `str` operations, `lambda` functions, `datetime` operations, and others. Choose the method that works best for you :)

In [23]:
#need to add format since it will default to 1970 dates
ins['date'] = pd.to_datetime(ins['date'], format='%Y%m%d')
ins['year'] = ins['date'].dt.year
print(ins)

       business_id  score       date     type  year
0               10     82 2016-05-03  routine  2016
1               10     94 2014-07-29  routine  2014
2               10     92 2014-01-14  routine  2014
3               19     94 2016-05-13  routine  2016
4               19     94 2014-11-10  routine  2014
...            ...    ...        ...      ...   ...
15425        89515     98 2016-11-03  routine  2016
15426        89701    100 2016-12-06  routine  2016
15427        89790     96 2016-11-29  routine  2016
15428        89803    100 2016-12-15  routine  2016
15429        89900     93 2016-12-06  routine  2016

[15430 rows x 5 columns]


#### Question 5d

What range of years is covered in this data set? Are there roughly same number of inspections each year? Try dropping records for any years with less than 50 inspections and store the result in a new dataframe named `ins3`.

In [24]:
#range of years covered in ins 2
ins['year'].value_counts()
print('The range of years covered in this data set is from 2013 to 2016.')

The range of years covered in this data set is from 2013 to 2016.


In [25]:
#are there roughly the same number of inspections each year?
ins['year'].value_counts()
print ('There are some years that have roughly the same number of inspections each year.')

There are some years that have roughly the same number of inspections each year.


In [26]:
ins3 = ins[ins['year'] > 2013]
print(ins3)


       business_id  score       date     type  year
0               10     82 2016-05-03  routine  2016
1               10     94 2014-07-29  routine  2014
2               10     92 2014-01-14  routine  2014
3               19     94 2016-05-13  routine  2016
4               19     94 2014-11-10  routine  2014
...            ...    ...        ...      ...   ...
15425        89515     98 2016-11-03  routine  2016
15426        89701    100 2016-12-06  routine  2016
15427        89790     96 2016-11-29  routine  2016
15428        89803    100 2016-12-15  routine  2016
15429        89900     93 2016-12-06  routine  2016

[15392 rows x 5 columns]


In [27]:
ins3['year'].value_counts()

2016    5840
2014    5629
2015    3923
Name: year, dtype: int64

Let's examine only the inspections for one year: 2016. This puts businesses on a more equal footing because [inspection guidelines](https://www.sfdph.org/dph/eh/Food/Inspections.asp) generally refer to how many inspections should occur in a given year.

In [28]:
ins3[ins3['year'] == 2016]

,business_id,score,date,type,year
0,10,82,2016-05-03,routine,2016
3,19,94,2016-05-13,routine,2016
6,24,98,2016-10-05,routine,2016
7,24,96,2016-03-11,routine,2016
10,31,84,2016-08-16,routine,2016
...,...,...,...,...,...
15425,89515,98,2016-11-03,routine,2016
15426,89701,100,2016-12-06,routine,2016
15427,89790,96,2016-11-29,routine,2016
15428,89803,100,2016-12-15,routine,2016


### Question 6

#### Question 6a

Merge the business and 2016 inspections data, keeping all businesses regardless of whether they show up in the inspections file. Show the first several rows of the resulting dataframe.

In [29]:
ins_2016 = ins3[ins3['year'] == 2016]
pd.merge(bus, ins_2016, on="business_id", how="outer")

,business_id,name,address,city,state,postal_code,latitude,longitude,phone_number,score,date,type,year
0,10,TIRAMISU KITCHEN,033 BELDEN PL,San Francisco,CA,94104,37.791116,-122.403816,+14154217044,82.0,2016-05-03,routine,2016.0
1,19,NRGIZE LIFESTYLE CAFE,"1200 VAN NESS AVE, 3RD FLOOR",San Francisco,CA,94109,37.786848,-122.421547,+14157763262,94.0,2016-05-13,routine,2016.0
2,24,OMNI S.F. HOTEL - 2ND FLOOR PANTRY,"500 CALIFORNIA ST, 2ND FLOOR",San Francisco,CA,94104,37.792888,-122.403135,+14156779494,98.0,2016-10-05,routine,2016.0
3,24,OMNI S.F. HOTEL - 2ND FLOOR PANTRY,"500 CALIFORNIA ST, 2ND FLOOR",San Francisco,CA,94104,37.792888,-122.403135,+14156779494,96.0,2016-03-11,routine,2016.0
4,31,NORMAN'S ICE CREAM AND FREEZES,2801 LEAVENWORTH ST,San Francisco,CA,94133,37.807155,-122.419004,NaN,84.0,2016-08-16,routine,2016.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7607,90234,ACCESS SFUSD THE ARC,1520 HOWARD ST,San Francisco,CA,94103,NaN,NaN,NaN,NaN,NaT,NaN,NaN
7608,90236,PHO DONG HUONG,667 MONTEREY BLVD,San Francisco,CA,94127,NaN,NaN,+14154945048,NaN,NaT,NaN,NaN
7609,90237,WIN'S RESTAURANT,3040 TARAVAL ST,San Francisco,CA,94116,NaN,NaN,+14158606036,NaN,NaT,NaN,NaN
7610,90238,USA POWER MARKET,1524 OCEAN AVE,San Francisco,CA,94112,NaN,NaN,+14153331878,NaN,NaT,NaN,NaN


#### Question 6b
Print the 20 lowest rated businesses names, their addresses, and their ratings.

In [30]:
#printing the 20 lowest score
bus_in = pd.merge(bus, ins_2016, on="business_id", how="outer")
bus_in2 = bus_in.sort_values(['score'], ascending=True).head(20)
bus_in3 = bus_in2[['name','address','score']].head(20)
print(bus_in3)

                             name              address  score
4892      GOLDEN RIVER RESTAURANT     5827 GEARY BLVD    52.0
7325                    POKI TIME     2101 LOMBARD ST    55.0
6281                 CRAZY PEPPER   2257 SAN JOSE AVE    55.0
4873                   UNCLE CAFE       65 WAVERLY PL    55.0
3008             HING WANG BAKERY        339 JUDAH ST    56.0
5627                   GOLDEN WOK   295 B ORIZABA AVE    56.0
5383                   TOP'S CAFE  2460 SAN BRUNO AVE    57.0
6067    L & G VIETNAMESE SANDWICH         602 EDDY ST    57.0
836                  RED A BAKERY      634 CLEMENT ST    57.0
4209      SAN TUNG RESTAURANT LLC      1031 IRVING ST    57.0
549           MA'S DIM SUM & CAFE      1315 POWELL ST    57.0
4903  NEW GARDEN RESTAURANT, INC.       716 KEARNY ST    57.0
6645    KOBANI MEDITERANEAN GRILL       3242 SCOTT ST    58.0
624           JOANIE'S DINER INC.   1329 COLUMBUS AVE    59.0
6938         PAK NAM THAI CUISINE       655 LARKIN ST    59.0
3563    

## Done!

Now commit this notebook to your `hw3_submissions` branch, push it to your GitHub repo, and open a PR!